In [ ]:
'''This will scrape Riyadh-local Twitter data over time'''

#import libraries and personal twitter API key
import twython
import json
import time
import threading
import pandas as pd
# import numpy as np

from datetime import datetime
from twython import Twython

from twitter_key import t_key, t_secret


In [ ]:
# Assigns the keys to the variables
APP_KEY = t_key
APP_SECRET = t_secret

# Fetches tweets with a given query at a given lat-long.
def get_tweets( latlong=None ):
    # Creates a Twython object with the given keys
    twitter = Twython( APP_KEY, APP_SECRET )
    # Uses the search function to hit the APIs endpoints and look for recent tweets within the area
    results = twitter.search( geocode=','.join([ str(x) for x in latlong ]) + ',15km', result_type='recent', count=10000)
    # Returns the only the statuses from the resulting JSON
    return results['statuses']

In [ ]:
''' Repeated collecting of tweets'''
def get_lots_of_tweets( latlong ):
    # Create a dictionary to parse the JSON
    all_tweets = {}
    
    # We will be hitting the API a number of times within the total time
    total_time = 120
    # Everytime we hit the API we subtract time from the total
    remaining_seconds = total_time
    interval = 30 
    while remaining_seconds > 0:
        added = 0
        # We hit the Twitter API
        new_tweets = get_tweets( latlong )
        # We parse the resulting JSON, and save the rest of the raw content
        for tweet in new_tweets:
            tid = tweet['id']
            if tid not in all_tweets and tweet['coordinates'] != None:
                properties = {}
                properties['lat'] = tweet['coordinates']['coordinates'][1]
                properties['lon'] = tweet['coordinates']['coordinates'][0]
                properties['tweet_id'] = tid
                properties['content'] = tweet['text']
                properties['user'] = tweet['user']['id']
                properties['user_location'] = tweet['user']['location']
                properties['raw_source'] = tweet
                properties['data_point'] = 'none'
                properties['time'] = tweet['created_at']
                all_tweets[ tid ] = properties
                added += 1
        print "At %d seconds, added %d new tweets, for a total of %d" % ( total_time - remaining_seconds, added, len( all_tweets ) )
        # We wait a few seconds and hit the API again
        time.sleep(interval)
        remaining_seconds -= interval
    # We return the final dictionary
    return all_tweets

In [ ]:
'''Define a centroid in Riaydh and grab a dictionary of sample tweets'''

latlong = [24.6333, 46.7167]
tweet_sample = get_lots_of_tweets( latlong )

In [ ]:
'''add it to a dataframe and export to csv'''
tweet_sample_locs = pd.DataFrame(tweet_sample)
tweet_sample_locs = tweet_sample_locs.transpose()

# tweet_sample_locs.head()
tweet_sample_locs.to_csv('sample_locs.csv', encoding='utf-8')